In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.appName("MobileLogs").getOrCreate()


25/05/13 10:27:03 WARN Utils: Your hostname, brempong-HP-EliteBook-840-G7-Notebook-PC resolves to a loopback address: 127.0.1.1; using 192.168.36.94 instead (on interface wlp0s20f3)
25/05/13 10:27:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/13 10:27:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [5]:
data = spark.read.csv('../Data/mobile-logs.csv', inferSchema=True, header=True)
data.show()

+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+
|               hour|     lat|   long|signal| network|   operator|status|     description|speed|satellites|precission|provider|activity|postal_code|
+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+
|2025-05-13 00:01:16|41.67089|0.53407|     2|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       4.0|      51.0|     gps|   STILL|   250236.0|
|2025-05-13 00:01:47|41.71431|1.77029|     3|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       2.0|     128.0|     gps|   STILL|    81786.0|
|2025-05-13 00:01:59|41.44174|1.86907|    11|  orange|    JAZZTEL|     0|STATE_IN_SERVICE|  0.2|       5.0|      25.0|     gps| TILTING|    80919.0|
|2025-05-13 00:05:15|41.20896|1.55499|     6|  orange|     Orange|     0|STATE_IN_SERVICE|  0.1|       5.0

In [6]:
cleaned_data = data.filter(
    col('hour').isNotNull() &
    col('lat').isNotNull() &
    col('long').isNotNull() &
    col('signal').isNotNull() &
    col('network').isNotNull() &
    col('operator').isNotNull() &
    col('status').isNotNull() &
    col('description').isNotNull() &
    col('speed').isNotNull() &
    col('satellites').isNotNull() &
    col('precission').isNotNull() &
    col('provider').isNotNull() &
    col('activity').isNotNull() &
    col('postal_code').isNotNull())

cleaned_data.show()

+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+
|               hour|     lat|   long|signal| network|   operator|status|     description|speed|satellites|precission|provider|activity|postal_code|
+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+
|2025-05-13 00:01:16|41.67089|0.53407|     2|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       4.0|      51.0|     gps|   STILL|   250236.0|
|2025-05-13 00:01:47|41.71431|1.77029|     3|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       2.0|     128.0|     gps|   STILL|    81786.0|
|2025-05-13 00:01:59|41.44174|1.86907|    11|  orange|    JAZZTEL|     0|STATE_IN_SERVICE|  0.2|       5.0|      25.0|     gps| TILTING|    80919.0|
|2025-05-13 00:05:15|41.20896|1.55499|     6|  orange|     Orange|     0|STATE_IN_SERVICE|  0.1|       5.0

In [ ]:
null_counts = cleaned_data.select([
    sum(col(c).isNull().cast("int")).alias(f"null_count_{c}") 
    for c in cleaned_data.columns
])
null_counts.show()

+---------------+--------------+---------------+-----------------+------------------+-------------------+-----------------+----------------------+----------------+---------------------+---------------------+-------------------+-------------------+----------------------+
|null_count_hour|null_count_lat|null_count_long|null_count_signal|null_count_network|null_count_operator|null_count_status|null_count_description|null_count_speed|null_count_satellites|null_count_precission|null_count_provider|null_count_activity|null_count_postal_code|
+---------------+--------------+---------------+-----------------+------------------+-------------------+-----------------+----------------------+----------------+---------------------+---------------------+-------------------+-------------------+----------------------+
|              0|             0|              0|                0|                 0|                  0|                0|                     0|               0|                    0|  

In [8]:
null_counts = data.select([
    sum(col(c).isNull().cast("int")).alias(f"null_count_{c}") 
    for c in data.columns
])
null_counts.show()

+---------------+--------------+---------------+-----------------+------------------+-------------------+-----------------+----------------------+----------------+---------------------+---------------------+-------------------+-------------------+----------------------+
|null_count_hour|null_count_lat|null_count_long|null_count_signal|null_count_network|null_count_operator|null_count_status|null_count_description|null_count_speed|null_count_satellites|null_count_precission|null_count_provider|null_count_activity|null_count_postal_code|
+---------------+--------------+---------------+-----------------+------------------+-------------------+-----------------+----------------------+----------------+---------------------+---------------------+-------------------+-------------------+----------------------+
|              0|             0|              0|                0|                 1|                  1|                0|                     0|               0|                    0|  

In [9]:
cleaned_data = cleaned_data.withColumn("year", year(col("hour"))) \
                        .withColumn("month", month(col("hour"))) \
                        .withColumn("day", dayofmonth(col("hour"))) \
                        .withColumn("hour_of_day", hour(col("hour")))
cleaned_data.show()

+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+----+-----+---+-----------+
|               hour|     lat|   long|signal| network|   operator|status|     description|speed|satellites|precission|provider|activity|postal_code|year|month|day|hour_of_day|
+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+----+-----+---+-----------+
|2025-05-13 00:01:16|41.67089|0.53407|     2|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       4.0|      51.0|     gps|   STILL|   250236.0|2025|    5| 13|          0|
|2025-05-13 00:01:47|41.71431|1.77029|     3|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       2.0|     128.0|     gps|   STILL|    81786.0|2025|    5| 13|          0|
|2025-05-13 00:01:59|41.44174|1.86907|    11|  orange|    JAZZTEL|     0|STATE_IN_SERVICE|  0.2|       5.0|      25.0|  

In [10]:
cleaned_data = cleaned_data.withColumn(
    "time", date_format(col("hour"), "HH:mm:ss")
)
cleaned_data.show()

+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+----+-----+---+-----------+--------+
|               hour|     lat|   long|signal| network|   operator|status|     description|speed|satellites|precission|provider|activity|postal_code|year|month|day|hour_of_day|    time|
+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+----+-----+---+-----------+--------+
|2025-05-13 00:01:16|41.67089|0.53407|     2|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       4.0|      51.0|     gps|   STILL|   250236.0|2025|    5| 13|          0|00:01:16|
|2025-05-13 00:01:47|41.71431|1.77029|     3|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       2.0|     128.0|     gps|   STILL|    81786.0|2025|    5| 13|          0|00:01:47|
|2025-05-13 00:01:59|41.44174|1.86907|    11|  orange|    JAZZTEL|     0|ST

In [11]:
cleaned_data = cleaned_data.withColumnRenamed("hour", "timestamp")
cleaned_data.show()

+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+----+-----+---+-----------+--------+
|          timestamp|     lat|   long|signal| network|   operator|status|     description|speed|satellites|precission|provider|activity|postal_code|year|month|day|hour_of_day|    time|
+-------------------+--------+-------+------+--------+-----------+------+----------------+-----+----------+----------+--------+--------+-----------+----+-----+---+-----------+--------+
|2025-05-13 00:01:16|41.67089|0.53407|     2|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       4.0|      51.0|     gps|   STILL|   250236.0|2025|    5| 13|          0|00:01:16|
|2025-05-13 00:01:47|41.71431|1.77029|     3|vodafone|vodafone ES|     0|STATE_IN_SERVICE|  0.0|       2.0|     128.0|     gps|   STILL|    81786.0|2025|    5| 13|          0|00:01:47|
|2025-05-13 00:01:59|41.44174|1.86907|    11|  orange|    JAZZTEL|     0|ST

In [12]:
cleaned_data.printSchema()

root
 |-- timestamp: timestamp (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- signal: integer (nullable = true)
 |-- network: string (nullable = true)
 |-- operator: string (nullable = true)
 |-- status: integer (nullable = true)
 |-- description: string (nullable = true)
 |-- speed: double (nullable = true)
 |-- satellites: double (nullable = true)
 |-- precission: double (nullable = true)
 |-- provider: string (nullable = true)
 |-- activity: string (nullable = true)
 |-- postal_code: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour_of_day: integer (nullable = true)
 |-- time: string (nullable = true)



In [ ]:
# KPI 1: Average Signal Strength and GPS Precision per Operator
operator_metrics = cleaned_data.groupBy("operator").agg(
    round(avg("signal").cast("double"), 1).alias("avg_signal_strength"),
    round(avg("precission"), 1).alias("avg_gps_precision")
)

operator_metrics.show()

+--------------------+-------------------+-----------------+
|            operator|avg_signal_strength|avg_gps_precision|
+--------------------+-------------------+-----------------+
|              Tuenti|                3.0|             68.4|
|               Yoigo|               12.2|             11.7|
|              Orange|               11.8|             34.7|
|           pepephone|               11.8|             34.1|
|            movistar|                8.5|             22.4|
|            Movistar|                9.2|             25.0|
|        Eroski Movil|               11.3|             14.7|
|               YOIGO|               14.5|             44.4|
|         vodafone ES|               12.8|             31.7|
|Solo llamadas de ...|               17.8|             17.3|
|             JAZZTEL|               11.0|             18.8|
|                RACC|                6.5|             22.6|
|               simyo|               31.5|             24.5|
+--------------------+--

In [ ]:
# KPI 2: Count of Network Statuses per Postal Code
# Use status_str if categorical mapping is applied, else use status
status_counts = cleaned_data.groupBy("postal_code").agg(
    count("*").alias("status_count")
)
status_counts.show()

+-----------+------------+
|postal_code|status_count|
+-----------+------------+
|   172142.0|           1|
|   250740.0|           2|
|    80351.0|           1|
|   430385.0|           1|
|    82212.0|           1|
|   172305.0|           1|
|    81568.0|           9|
|   171479.0|           3|
|    80039.0|          10|
|    81043.0|           6|
|   172076.0|           1|
|    82359.0|           1|
|    81803.0|           1|
|    82613.0|           5|
|   250030.0|           7|
|   170427.0|           4|
|    81311.0|           1|
|   171143.0|           6|
|    82690.0|          14|
|    80898.0|           2|
+-----------+------------+
only showing top 20 rows

